In [1]:
import os


In [2]:
%pwd

'c:\\Users\\Saura\\OneDrive\\Desktop\\Upskilling-Reskilling\\Python and ML\\Projects\\NLP_Txt_Summarizer\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Saura\\OneDrive\\Desktop\\Upskilling-Reskilling\\Python and ML\\Projects\\NLP_Txt_Summarizer\\Text-Summarizer-Project'

In [12]:
##src/textSummarizer/entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [13]:
#src/textSummarizer/sconfig
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config


In [23]:
#src/textSummarizer/components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                self.config.source_url,
                filename=self.config.local_data_file)
            logger.info(f"Downloaded {filename} from {self.config.source_url}")
     
        else:
            logger.info(f"File of size : {get_size(Path(self.config.local_data_file))} already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [27]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    logger.info("Data ingestion completed successfully")

except Exception as e:
    logger.error(f"An error occurred during data ingestion: {str(e)}")
    raise e

[2024-07-05 23:55:53,783: INFO: common 23 yaml file : config\config.yaml loaded successfully]
[2024-07-05 23:55:53,791: INFO: common 23 yaml file : params.yaml loaded successfully]
[2024-07-05 23:55:53,796: INFO: common 42 Directory created : artifacts]
[2024-07-05 23:55:53,801: INFO: common 42 Directory created : artifacts\data_ingestion]
[2024-07-05 23:56:11,470: INFO: 3667595434 10 Downloaded artifacts/data_ingestion/data.zip from https://github.com/SauravKPandey/Datasets/raw/main/summarizer-data.zip]
[2024-07-05 23:56:12,990: INFO: 2058472703 7 Data ingestion completed successfully]
